In [1]:
import pandas as pd
import polars as pl
import re
from api_scraper1 import MLB_Scrape

# Initialize the scraper
scraper = MLB_Scrape()

In [2]:
df_player = scraper.get_players(sport_id=1,season=2025,game_type=['S'])
name = 'Jack Leiter'  # Replace with the name you're looking for
specific_player = df_player.filter(df_player['name'] == name)
print(specific_player)

shape: (1, 6)
┌───────────┬────────────┬───────────┬─────────────┬──────────┬──────┐
│ player_id ┆ first_name ┆ last_name ┆ name        ┆ position ┆ team │
│ ---       ┆ ---        ┆ ---       ┆ ---         ┆ ---      ┆ ---  │
│ i64       ┆ str        ┆ str       ┆ str         ┆ str      ┆ i64  │
╞═══════════╪════════════╪═══════════╪═════════════╪══════════╪══════╡
│ 683004    ┆ Jack       ┆ Leiter    ┆ Jack Leiter ┆ P        ┆ 140  │
└───────────┴────────────┴───────────┴─────────────┴──────────┴──────┘


In [4]:
import polars as pl

# Set player ID and the desired date
player_id = 683004
selected_date = '2025-03-16'  # Replace with your desired date (format: 'YYYY-MM-DD')
season = 2025
player_games = scraper.get_player_games_list(player_id=player_id, season=season, game_type=['S'], pitching=True)

# Get the data using the game list
data = scraper.get_data(game_list_input=player_games)

# Convert the data into a Polars DataFrame
df = scraper.get_data_df(data_list=data)

# Filter the DataFrame based on the player ID and selected date
df = df.filter(
        (pl.col("pitcher_id") == player_id) & 
        (pl.col("game_date") == selected_date)  # Filter for the specific date
)

# Convert the Polars DataFrame to a Pandas DataFrame
df = df.to_pandas()
pitcher_id = player_id
# Now df_pandas is a Pandas DataFrame and you can use it for further analysis or plotting
print(df.columns)
# Display the first few rows of release position columns
print(df[['release_pos_x', 'release_pos_z', 'z_release_point', 'x', 'y']].head(10))



This May Take a While. Progress Bar shows Completion of Data Retrieval.


Processing: 100%|██████████| 5/5 [00:00<00:00, 16.36iteration/s]

Converting Data to Dataframe.
Index(['game_id', 'game_date', 'batter_id', 'batter_name', 'batter_hand',
       'batter_team', 'batter_team_id', 'pitcher_id', 'pitcher_name',
       'pitcher_hand', 'pitcher_team', 'pitcher_team_id', 'ab_number',
       'play_description', 'play_code', 'in_play', 'is_strike', 'is_swing',
       'is_whiff', 'is_out', 'is_ball', 'is_review', 'pitch_type',
       'pitch_description', 'strikes', 'balls', 'outs', 'strikes_after',
       'balls_after', 'outs_after', 'start_speed', 'end_speed', 'sz_top',
       'sz_bot', 'x', 'y', 'ax', 'ay', 'az', 'pfxx', 'pfxz', 'px', 'pz', 'vx0',
       'vy0', 'vz0', 'x0', 'y0', 'z0', 'zone', 'type_confidence', 'plate_time',
       'extension', 'spin_rate', 'spin_direction', 'vb', 'ivb', 'hb',
       'release_pos_x', 'release_pos_z', 'z_release_point', 'launch_speed',
       'launch_angle', 'launch_distance', 'launch_location', 'trajectory',
       'hardness', 'hit_x', 'hit_y', 'index_play', 'play_id', 'start_time',
       '

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming 'df' is your dataframe with release position data for the pitcher
# Filter the data for the specific pitcher
df_pitcher = df[df['pitcher_id'] == pitcher_id]  # Replace 'pitcher_id' with your pitcher ID

# Calculate the average release positions
avg_release_x = np.mean(df_pitcher['release_pos_x'])
avg_release_y = np.mean(df_pitcher['release_pos_y'])

# Create a plot
fig, ax = plt.subplots(figsize=(8, 6))

# Scatter plot for individual release positions
sns.scatterplot(data=df_pitcher, x='release_pos_x', y='release_pos_y', ax=ax, color='blue', alpha=0.6)

# Plot the average release point
ax.scatter(avg_release_x, avg_release_y, color='red', marker='X', s=100, label=f'Average Release\n({avg_release_x:.2f}, {avg_release_y:.2f})')

# Add labels and title
ax.set_xlabel('Release Position X (Horizontal) [inches]', fontsize=12)
ax.set_ylabel('Release Position Y (Vertical) [inches]', fontsize=12)
ax.set_title('Pitcher Release Points (Average)', fontsize=14)

# Show the legend
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()


KeyError: 'release_pos_y'